In [616]:
import pandas as pd
import numpy as np
from pymoo.optimize import minimize
import pickle

from pymoo.util.termination.f_tol import MultiObjectiveSpaceToleranceTermination
from pymoo.visualization.scatter import Scatter
import contact
import util
from notebooks.optimization_problems.constraints import Requirements
from pyreport import PlotUtil
import matplotlib.pyplot as plt

import notebooks.util

%matplotlib

plt.close('all')

%load_ext autoreload
%autoreload 2

Using matplotlib backend: Qt5Agg
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [617]:
prefix = 'iot_' + 'o3b_'

case, target = pickle.load(open('raw/%ssettings.pkl' % prefix, 'rb'))

import util
util.print_targets(case)

Targets per orbit
T_target: 20.00 Gb (min: 20.00 Gb, max: -0.00 Gb)
L_target: 60.00 s
E_target: 16.82 kJ (max: -0.00 kJ)
P_target: 0.00 s


In [618]:
columns = ['strand_name', 'tof',
           'r_a_x', 'r_a_y', 'r_a_z',
           'v_a_x', 'v_a_y', 'v_a_z',
           'r_b_x', 'r_b_y', 'r_b_z',
           'd',
           'r_ab_sff_x', 'r_ab_sff_y', 'r_ab_sff_z']

file_path = "raw/%spost_processed.h5" % prefix

row_limit = -1

hdf_store = pd.HDFStore(file_path)

instances_df = hdf_store.select('contact_instances', 'columns = %s' % str(columns), stop = row_limit)

# Sort by pass id
instances_df = instances_df.sort_index(0)

In [619]:
plt.close('Performance')

# Plot the optimizations
fig = plt.figure(num='Performance', figsize=(3.2*3, 2.4*2))
axs = fig.subplots(2,3, gridspec_kw={'height_ratios': [1, 1]})
axs = axs.flatten()

N_orbits = case['T_sim_s'] / case['T_orbit_s']
scale_factors = np.array([1/-1e9, 1, 1/1e3, 1]) * \
                np.array([1/N_orbits, 1, 1/N_orbits, 1/case['T_sim_s']]) * \
                np.array([1/8, 1, 1, 100])

problem, setting, res = pickle.load(open('raw/iot_o3b_unsga3_xover_300_300_75.pkl', 'rb'))
print(res.algorithm.n_gen)
util.plot_performance_iot(axs, problem, setting, res, case, target, scale_factors,
                          color='tab:grey', alpha=1, throughput=False, plot_i=False)
axs[5].scatter(np.NaN, np.NaN, marker='.', color='tab:grey', s=1, label='U-NSGA-3 4500 generations')

4500


No handles with labels found to put in legend.


In [620]:
selection = 'raw/iot_o3b_unsga3_xover_300_300_75.pkl'

points = []

# Targets
p = np.array([case['T_bitorbit_target'], case['L_sorbit_target'], case['E_Jorbit_target'], case['P_sorbit_target']/3])
p = p * np.array([-1, 1, 1, 1]) * np.array([N_orbits, 1, N_orbits, N_orbits])
p_fmt = p * scale_factors
points.append({'ind': -1, 'p': p, 'args': [], 'kwargs': {'s': 20, 'color': 'r', 'marker': 'x', 'label': 'Target\n(T=%0.1f GB/orbit, L=%d s, E=%d kJ, P=%0.2f %%)' % tuple(p_fmt)}})

# # Closest to target
# problem, setting, res = pickle.load(open(selection, 'rb'))
# ref = np.array([60, case['E_Jorbit_target'] * N_orbits])
# diff= np.abs(res.F[:, [0, 1]] - ref) / ref
# ind_close = np.argmin(np.linalg.norm(diff, axis=1))
# p = np.array(util.recompute_obj(problem, res, scale_factors=1))
# p = p[:, ind_close]
# p_fmt = p * scale_factors
# points.append({'ind': ind_close, 'p': p, 'args': [], 'kwargs': {'s': 25, 'color': 'b', 'marker': '+', 'label': 'Closest\n(T=%0.1f GB/orbit, L=%d s, E=%d kJ, P=%0.2f %%)' % tuple(p_fmt)}})

# Select point
problem, setting, res = pickle.load(open(selection, 'rb'))
ind_select = 33
p = np.array(util.recompute_obj(problem, res, scale_factors=1))
p = p[:, ind_select]
p_fmt = p * scale_factors
points.append({'ind': ind_select, 'p': p, 'args': [], 'kwargs': {'s': 25, 'color': 'g', 'marker': '*', 'label': '35 dBi antenna\n(T=%0.1f GB/orbit, L=%d s, E=%d kJ, P=%0.2f %%)' % tuple(p_fmt)}})

# Select point
problem, setting, res = pickle.load(open(selection, 'rb'))
ind_select = 49
p = np.array(util.recompute_obj(problem, res, scale_factors=1))
p = p[:, ind_select]
p_fmt = p * scale_factors
points.append({'ind': ind_select, 'p': p, 'args': [], 'kwargs': {'s': 25, 'color': 'b', 'marker': '+', 'label': '50 dBi antenna\n(T=%0.1f GB/orbit, L=%d s, E=%d kJ, P=%0.2f %%)' % tuple(p_fmt)}})

fig = plt.figure(num="Performance")
util.plot_points_iot(axs, points, scale_factors=scale_factors)
plt.savefig('D:/git/thesis_report_ae/figures/relay_optimization/%sresult.pdf' % prefix)

In [ ]:
plt.close('Settings')

fig = plt.figure(num='Settings', figsize=(3.2*3, 2.4*3))
axs = fig.subplots(3,3, gridspec_kw={'height_ratios': [1, 1, 1]})
axs = axs.flatten()

problem, setting, res = pickle.load(open(selection, 'rb'))
#problem, setting, res = pickle.load(open('raw/iot_telesat_rnsga2_1e-2_cont_300_75.pkl', 'rb'))
util.plot_settings(axs, problem, setting, res, points=points, scale_factors=scale_factors)

In [ ]:
util.plot_power_energy(case, instances_df, problem, res.X[ind_close,:])
plt.savefig('D:/git/thesis_report_ae/figures/relay_optimization/%spower_energy.pdf' % prefix)
pass

bbox = plt.gca().get_position()

In [ ]:
util.plot_used_passes(case, instances_df, problem, res.X[ind_close,:])
plt.gca().set_position(bbox)

plt.savefig('D:/git/thesis_report_ae/figures/relay_optimization/%srange_selection.pdf' % prefix)

In [ ]:
util.plot_pointing(case, problem, res.X[ind_close,:])
plt.gca().set_position(bbox)

plt.savefig('D:/git/thesis_report_ae/figures/relay_optimization/%spointing.pdf' % prefix)

In [ ]:
hdf_store.close()